In [4]:
#model 1

using JuMP
using Gurobi
using DataFrames

m = Model()

@variable(m, Sup[i = 1:10, j = 1:5],Bin)

df = readtable("/Users/lyynikki/Desktop/1.csv")
cost_raw = df[:Cost_NT_]
ul = df[:UL_Certified]
extra_cost = df[:Extra_Pre_Assembly_Cost_NT_]
defect_raw = df[:Defect_Rate]
total_cost_unshaped = [(cost_raw[i]+extra_cost[i]) *(1+defect_raw[i]) for i=1:43]

UL = zeros(10,5)
UL[1,1:4] = ul[1:4]
UL[2,1:5] = ul[5:9]
UL[3,1:4] = ul[10:13]
UL[4,1:5] = ul[14:18]
UL[5,1:4] = ul[19:22]
UL[6,1:4] = ul[23:26]
UL[7,1:4] = ul[27:30]
UL[8,1:5] = ul[31:35]
UL[9,1:5] = ul[36:40]
UL[10,1:3] = ul[41:43]
UL

Cost = zeros(10,5)
Cost[1,1:4] = total_cost_unshaped[1:4]
Cost[2,1:5] = total_cost_unshaped[5:9]
Cost[3,1:4] = total_cost_unshaped[10:13]
Cost[4,1:5] = total_cost_unshaped[14:18]
Cost[5,1:4] = total_cost_unshaped[19:22]
Cost[6,1:4] = total_cost_unshaped[23:26]
Cost[7,1:4] = total_cost_unshaped[27:30]
Cost[8,1:5] = total_cost_unshaped[31:35]
Cost[9,1:5] = total_cost_unshaped[36:40]
Cost[10,1:3] = total_cost_unshaped[41:43]
Cost

Defect = zeros(10,5)
Defect[1,1:4] = defect_raw[1:4]
Defect[2,1:5] = defect_raw[5:9]
Defect[3,1:4] = defect_raw[10:13]
Defect[4,1:5] = defect_raw[14:18]
Defect[5,1:4] = defect_raw[19:22]
Defect[6,1:4] = defect_raw[23:26]
Defect[7,1:4] = defect_raw[27:30]
Defect[8,1:5] = defect_raw[31:35]
Defect[9,1:5] = defect_raw[36:40]
Defect[10,1:3] = defect_raw[41:43]
Defect

@objective(m, Min, sum{Cost[i,j]*Sup[i,j], i=1:10, j=1:5})
for i=1:10
    @constraint(m, sum{Sup[i, j], j = 1:5} == 1)
end

@constraint(m, Sup[1,3]+Sup[2,3]<=1)
@constraint(m, sum{UL[i,j]*Sup[i,j],i = 1:10, j = 1:5} >= 7)
@constraint(m,Sup[2,1]== Sup[4,2])
@constraint(m,Sup[1,5] == Sup[3,5])
@constraint(m,Sup[5,5] == Sup[6,5])
@constraint(m,Sup[7,5] == Sup[10,5])
@constraint(m,Sup[10,4] == 0)
@constraint(m,Sup[1,5] == 0)
@constraint(m,Sup[5,5] == 0)
@constraint(m,Sup[7,5] == 0)

for i=1:10
@constraint(m,sum{ExtraCost[i,j]*Sup[i,j], j=1:5}+sum{Cost[i,j]*Sup[i,j], j=1:5}<= 5400*33.43)
end

status = solve(m)
println("Objective value is ", getobjectivevalue(m) )
getvalue(Sup)



10×5 Array{Float64,2}:
 0.037  0.017  0.015  0.018  0.0  
 0.043  0.013  0.06   0.026  0.024
 0.034  0.1    0.011  0.091  0.0  
 0.031  0.012  0.021  0.02   0.016
 0.068  0.085  0.075  0.067  0.0  
 0.014  0.014  0.061  0.02   0.0  
 0.016  0.013  0.03   0.057  0.0  
 0.035  0.058  0.063  0.046  0.014
 0.019  0.029  0.059  0.091  0.035
 0.077  0.031  0.038  0.0    0.0  

In [5]:
# model 1 cost
x = getvalue(Sup)
Cost.*x
sum(Cost.*x)
# model 1 prob that all are not replaced
y = ones(10,5)
x.*(y .- Defect)
# model 1 prob that at most 1 need to be replaced 
a = (Defect ./(y .- Defect)).*x*0.6087
sum(a)+0.6087

Optimize a model with 30 rows, 50 columns and 138 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 2e+03]
  Objective range [2e+02, 2e+03]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 2e+05]
Found heuristic solution: objective 5598.58
Presolve removed 30 rows and 50 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 5.117103000000e+03, best bound 5.117103000000e+03, gap 0.0%
Objective value is 5117.103


10×5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0

In [48]:
# model 2

using JuMP
using Gurobi
using DataFrames

m = Model()

@variable(m, Sup[i = 1:10, j = 1:5],Bin)

df = readtable("/Users/lyynikki/Desktop/1.csv")
cost_raw = df[:Cost_NT_]
ul = df[:UL_Certified]
extra_cost = df[:Extra_Pre_Assembly_Cost_NT_]
defect_raw = df[:Defect_Rate]
total_cost_unshaped = [(cost_raw[i]+extra_cost[i]) *(1+defect_raw[i]) for i=1:43]

UL = zeros(10,5)
UL[1,1:4] = ul[1:4]
UL[2,1:5] = ul[5:9]
UL[3,1:4] = ul[10:13]
UL[4,1:5] = ul[14:18]
UL[5,1:4] = ul[19:22]
UL[6,1:4] = ul[23:26]
UL[7,1:4] = ul[27:30]
UL[8,1:5] = ul[31:35]
UL[9,1:5] = ul[36:40]
UL[10,1:3] = ul[41:43]
UL

Cost = zeros(10,5)
Cost[1,1:4] = total_cost_unshaped[1:4]
Cost[2,1:5] = total_cost_unshaped[5:9]
Cost[3,1:4] = total_cost_unshaped[10:13]
Cost[4,1:5] = total_cost_unshaped[14:18]
Cost[5,1:4] = total_cost_unshaped[19:22]
Cost[6,1:4] = total_cost_unshaped[23:26]
Cost[7,1:4] = total_cost_unshaped[27:30]
Cost[8,1:5] = total_cost_unshaped[31:35]
Cost[9,1:5] = total_cost_unshaped[36:40]
Cost[10,1:3] = total_cost_unshaped[41:43]
Cost

Defect = zeros(10,5)
Defect[1,1:4] = defect_raw[1:4]
Defect[2,1:5] = defect_raw[5:9]
Defect[3,1:4] = defect_raw[10:13]
Defect[4,1:5] = defect_raw[14:18]
Defect[5,1:4] = defect_raw[19:22]
Defect[6,1:4] = defect_raw[23:26]
Defect[7,1:4] = defect_raw[27:30]
Defect[8,1:5] = defect_raw[31:35]
Defect[9,1:5] = defect_raw[36:40]
Defect[10,1:3] = defect_raw[41:43]
Defect


@objective(m, Min, sum{Defect[i,j]*Sup[i,j], i=1:10, j=1:5})
for i=1:10
    @constraint(m, sum{Sup[i, j], j = 1:5} == 1)
end

@constraint(m, Sup[1,3]+Sup[2,3]<=1)
@constraint(m, sum{UL[i,j]*Sup[i,j],i = 1:10, j = 1:5} >= 7)
@constraint(m,Sup[2,1]== Sup[4,2])
@constraint(m,Sup[1,5] == Sup[3,5])
@constraint(m,Sup[5,5] == Sup[6,5])
@constraint(m,Sup[7,5] == Sup[10,5])
@constraint(m,Sup[10,4] == 0)
@constraint(m,Sup[1,5] == 0)
@constraint(m,Sup[5,5] == 0)
@constraint(m,Sup[7,5] == 0)

for i=1:10
@constraint(m,sum{ExtraCost[i,j]*Sup[i,j], j=1:5}+sum{Cost[i,j]*Sup[i,j], j=1:5}<= 5400*33.43)
end

status = solve(m)
println("Objective value is ", getobjectivevalue(m) )
getvalue(Sup)

10×5 Array{Float64,2}:
 0.0233872  0.0        0.0         0.0  0.0
 0.0        0.0        0.0388532   0.0  0.0
 0.0        0.0        0.00677017  0.0  0.0
 0.0        0.0        0.0130569   0.0  0.0
 0.0444116  0.0        0.0         0.0  0.0
 0.0        0.0        0.0395428   0.0  0.0
 0.0        0.0        0.0188258   0.0  0.0
 0.0        0.0374783  0.0         0.0  0.0
 0.0        0.0        0.038165    0.0  0.0
 0.05078    0.0        0.0         0.0  0.0

In [43]:
# model 2 cost
x = getvalue(Sup)
Cost.*x
sum(Cost.*x)
# model 2 prob that all are not replaced
y = ones(10,5)
x.*(y .- Defect)
# model 2 prob that at most 1 need to replace 
a = (Defect ./(y .- Defect)).*x*0.8052
sum(a)+0.8052

10×5 Array{Float64,2}:
 0.963  0.0    0.0    0.0  0.0
 0.0    0.0    0.94   0.0  0.0
 0.0    0.0    0.989  0.0  0.0
 0.0    0.0    0.979  0.0  0.0
 0.932  0.0    0.0    0.0  0.0
 0.0    0.0    0.939  0.0  0.0
 0.0    0.0    0.97   0.0  0.0
 0.0    0.942  0.0    0.0  0.0
 0.0    0.0    0.941  0.0  0.0
 0.923  0.0    0.0    0.0  0.0

In [47]:
# model 3
using JuMP
using Gurobi
using DataFrames

m = Model()

@variable(m, Sup[i = 1:10, j = 1:5],Bin)

df = readtable("/Users/lyynikki/Desktop/1.csv")
cost_raw = df[:Cost_NT_]
ul = df[:UL_Certified]
extra_cost = df[:Extra_Pre_Assembly_Cost_NT_]
defect_raw = df[:Defect_Rate]
total_cost_unshaped = [(cost_raw[i]+extra_cost[i])for i=1:43]

UL = zeros(10,5)
UL[1,1:4] = ul[1:4]
UL[2,1:5] = ul[5:9]
UL[3,1:4] = ul[10:13]
UL[4,1:5] = ul[14:18]
UL[5,1:4] = ul[19:22]
UL[6,1:4] = ul[23:26]
UL[7,1:4] = ul[27:30]
UL[8,1:5] = ul[31:35]
UL[9,1:5] = ul[36:40]
UL[10,1:3] = ul[41:43]
UL

Cost = zeros(10,5)
Cost[1,1:4] = total_cost_unshaped[1:4]
Cost[2,1:5] = total_cost_unshaped[5:9]
Cost[3,1:4] = total_cost_unshaped[10:13]
Cost[4,1:5] = total_cost_unshaped[14:18]
Cost[5,1:4] = total_cost_unshaped[19:22]
Cost[6,1:4] = total_cost_unshaped[23:26]
Cost[7,1:4] = total_cost_unshaped[27:30]
Cost[8,1:5] = total_cost_unshaped[31:35]
Cost[9,1:5] = total_cost_unshaped[36:40]
Cost[10,1:3] = total_cost_unshaped[41:43]
Cost

Defect = zeros(10,5)
Defect[1,1:4] = defect_raw[1:4]
Defect[2,1:5] = defect_raw[5:9]
Defect[3,1:4] = defect_raw[10:13]
Defect[4,1:5] = defect_raw[14:18]
Defect[5,1:4] = defect_raw[19:22]
Defect[6,1:4] = defect_raw[23:26]
Defect[7,1:4] = defect_raw[27:30]
Defect[8,1:5] = defect_raw[31:35]
Defect[9,1:5] = defect_raw[36:40]
Defect[10,1:3] = defect_raw[41:43]
Defect

@objective(m, Min, sum{Defect[i,j]*Sup[i,j]*Cost[i,j], i=1:10, j=1:5})
for i=1:10
    @constraint(m, sum{Sup[i, j], j = 1:5} == 1)
end

@constraint(m, Sup[1,3]+Sup[2,3]<=1)
@constraint(m, sum{UL[i,j]*Sup[i,j],i = 1:10, j = 1:5} >= 7)
@constraint(m,Sup[2,1]== Sup[4,2])
@constraint(m,Sup[1,5] == Sup[3,5])
@constraint(m,Sup[5,5] == Sup[6,5])
@constraint(m,Sup[7,5] == Sup[10,5])
@constraint(m,Sup[10,4] == 0)
@constraint(m,Sup[1,5] == 0)
@constraint(m,Sup[5,5] == 0)
@constraint(m,Sup[7,5] == 0)

for i=1:10
    @constraint(m,sum{(1+Defect[i,j])*Sup[i,j]*Cost[i,j],i=1:10, j=1:5}<= 5400*33.43)
end

status = solve(m)
println("Objective value is ", getobjectivevalue(m) )
getvalue(Sup)

0.9199709960719735

In [30]:
total_cost_unshaped = [(cost_raw[i]+extra_cost[i]) *(1+defect_raw[i]) for i=1:43]
Cost = zeros(10,5)
Cost[1,1:4] = total_cost_unshaped[1:4]
Cost[2,1:5] = total_cost_unshaped[5:9]
Cost[3,1:4] = total_cost_unshaped[10:13]
Cost[4,1:5] = total_cost_unshaped[14:18]
Cost[5,1:4] = total_cost_unshaped[19:22]
Cost[6,1:4] = total_cost_unshaped[23:26]
Cost[7,1:4] = total_cost_unshaped[27:30]
Cost[8,1:5] = total_cost_unshaped[31:35]
Cost[9,1:5] = total_cost_unshaped[36:40]
Cost[10,1:3] = total_cost_unshaped[41:43]
Cost

x = getvalue(Sup)
Cost.*x
sum(Cost.*x)

(0.037,0.06,0.011,0.021,0.068,0.061,0.03,0.058,0.059,0.077)

In [53]:
#model 3 prob 1
y = ones(10,5)
x.*(y .- Defect)
# model 3 prob 2
a = (Defect ./(y .- Defect)).*x*0.8043
sum(a) + 0.8043


0